Introduction
This notebook demonstrates how to build a deep learning model using ResNet50, a pre-trained model, to classify faces into three different age groups:

YOUNG
MIDDLE
OLD
The dataset used contains images of people, and the model is trained to categorize them into one of these three age groups. The notebook covers data preprocessing, model creation, training, evaluation, and testing.

Steps Overview
Loading the Dataset: Load the face images and corresponding age group labels.
Preprocessing: Resize the images and augment the data to improve model generalization.
Building the Model: Use the pre-trained ResNet50 model and add custom layers on top of it.
Training: Train the model using the preprocessed data.
Evaluation: Evaluate the model’s performance and visualize the results.
Testing: Use the model to predict the age group of new face images.
1. Loading the Dataset
We start by loading a CSV file that contains image file names and their associated age group. The age groups are mapped to numeric values:

YOUNG: 0
MIDDLE: 1
OLD: 2

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv(r"/kaggle/input/faces-age-detection-dataset/faces/train.csv")

# Map class labels to numeric values
age_group = {"YOUNG": 0, "MIDDLE": 1, "OLD": 2}
df['target'] = df['Class'].map(age_group)


2. Preprocessing
Image Resizing
Each image is resized to 180x180 pixels because the model expects all images to have the same dimensions. We also normalize pixel values to the range [0, 1].

Data Augmentation
To make the model more robust, we apply data augmentation using ImageDataGenerator. This includes transformations like:

Horizontal flipping (mirroring the image)
Brightness adjustment
Shifting the image to the left or right
Data augmentation helps the model generalize better by making it less dependent on the exact structure of the training data.

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Resize images
folder_path = '/kaggle/input/faces-age-detection-dataset/faces/Train'
df['image'] = df['ID'].apply(lambda id: cv2.resize(plt.imread(os.path.join(folder_path, id)), (180, 180)))

# Data Augmentation using ImageDataGenerator
imggen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    brightness_range=(0.4, 0.55),  # Adjust brightness
    horizontal_flip=True,  # Flip images horizontally
    width_shift_range=0.22  # Shift image left or right
)


Splitting the Data
The dataset is split into a training set and a validation set. The training set is used to train the model, while the validation set is used to evaluate the model's performance.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Shuffle the dataset
augmented_images = np.stack(df['image'].values)
augmented_labels = df['target'].values
aug_imgs, class_list = shuffle(augmented_images, augmented_labels, random_state=42)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(aug_imgs, class_list, test_size=0.2, random_state=42)


3. Building the Model
Using ResNet50
ResNet50 is a pre-trained deep learning model that has been trained on millions of images. Instead of training a model from scratch, we use ResNet50's knowledge and fine-tune it for our specific task of age group classification. We freeze the layers of ResNet50 so that they are not modified during training, and add new layers on top to make predictions for our three age groups.

Custom Layers
On top of ResNet50, we add the following layers:

Flatten: Converts the output of ResNet50 into a 1D array.
Dense Layers: Fully connected layers with ReLU activation to learn complex patterns.
Dropout: Helps prevent overfitting by randomly setting some layer outputs to zero during training.
Softmax Layer: Produces probabilities for the three age group classes